## !! The final result should be only a runnable .py file !!

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import pickle
from torch.utils.data import random_split
from models import model_1 as m
from training_early_stop import EarlyStop
import utility

# 0. Data Pre-processing

In [2]:
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # turn the graph to single color channel
    transforms.Resize((227, 227)), # resize to 227 * 227 because we use AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # normalize
])

train_dataset = datasets.ImageFolder(
    '../dataset/train', transform=data_transforms)
# split training set to training set and validation set
# a random seed to ensure reproducibility of results.
torch.manual_seed(42)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

test_dataset = datasets.ImageFolder('../dataset/test', transform=data_transforms)


train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=512,shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=8, pin_memory=False)

In [3]:
# select device
device = utility.select_devices()

using CUDA + cudnn


# 1. Model

In [4]:
# initialize model, loss-function and optimizer
model = m.EmotionCNN(num_classes=7)  # FER-2013 has 7 emotion class
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [5]:
# average loss / epoch
loss_history_per_epoch = []
# correct prediction / epoch
correct_prediction_pre_epoch = []
# accuracy / epoch
accuracy_per_epoch = []
# validation loss
val_loss_per_epoch = []
# validation accuracy
val_accuracy_per_epoch = []

In [6]:
# training model
num_epochs = 2000
model.to(device)
model.train()

# early stopping variables
count_to_stop = 10
stopping_count = 15
different = 0.001
interval = 30
counter = 0
is_always = False
is_exe = False
early_stopping = EarlyStop(m.pth_save_path, count_to_stop, different)

# progress bar
process = tqdm(range(num_epochs), bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}', colour='green', ascii='░▒█', unit='epoch')

for epoch in process:
    running_loss = 0.0
    accuracy = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # forwarding get output
        outputs = model(inputs)
        # compute loss of output
        loss = criterion(outputs, labels)
        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # record training status
        running_loss += loss.item()
        prediction = outputs.argmax(dim=1)
        num_correct_prediction = (prediction == labels).sum().item()
        correct_prediction_pre_epoch.append(num_correct_prediction)
        accuracy += num_correct_prediction / inputs.shape[0]
    # save training status
    loss_history_per_epoch.append((running_loss / len(train_loader)))
    accuracy_per_epoch.append((accuracy / len(train_loader)))

    # training validation + early stopping
    if is_always or is_exe or (epoch != 0 and epoch % interval == 0):
        val_loss = 0.0
        val_accuracy = 0.0

        if not is_always and epoch% interval ==0:
            early_stopping.counter = 0
            is_exe = True

        counter += 1

        if not is_always and counter >= stopping_count:
            counter = 0
            is_exe = False

        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            prediction = outputs.argmax(dim=1)
            num_correct_prediction = (prediction == labels).sum().item()
            accuracy = num_correct_prediction / inputs.shape[0]
            val_accuracy += accuracy
        val_loss_per_epoch.append((val_loss / len(val_loader)))
        val_accuracy_per_epoch.append((val_accuracy / len(val_loader)))

        early_stopping.check_status(model, val_loss)

        # display recently 5 average loss of epochs
        process.set_description(f"avg loss[-5:] = {loss_history_per_epoch[-5:]}\t"
                                f"accuracy[-5:] = {accuracy_per_epoch[-5:]}\t"
                                f"best loss = {early_stopping.min_loss}, val loss = {val_loss}\t"
                                f"val accuracy[-5] = {val_accuracy_per_epoch[-5:]}\t"
                                f"Counter = {early_stopping.counter}/{count_to_stop} | {counter}/{stopping_count}\t")
    else:
        process.set_description(f"avg loss[-5:] = {loss_history_per_epoch[-5:]}\t"
                                f"accuracy[-5:] = {accuracy_per_epoch[-5:]}\t")

    if early_stopping.early_stop:
        print('\nTrigger Early Stopping\n')
        break

avg loss[-5:] = [0.40504605107837255, 0.37902953624725344, 0.3570103645324707, 0.3431914422247145, 0.31357561813460455]	accuracy[-5:] = [0.8559690189192609, 0.8663059984813971, 0.8743499470070868, 0.8782634143254872, 0.8907558015375854]	best loss = 17.217087388038635, val loss = 20.40248191356659	val accuracy[-5] = [0.5759469696969697, 0.5756155303030303, 0.5673650568181818, 0.5642282196969697, 0.5791015625]	Counter = 10/10 | 11/15	:   2%|░░░░░░░░░░░░░░░░░░░░| 40/2000 [14:00<11:26:27, 21.01s/epoch]      


Trigger Early Stopping


In [7]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # predicted is the emotion index
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy}%")

Test Accuracy: 60.810810810810814%


In [8]:
# save the pth file
torch.save(model.state_dict(), m.pth_manual_save_path)

utility.save_pickle_files(loss_history_per_epoch, m.record_save_path + '/loss_history.pkl')
utility.save_pickle_files(accuracy_per_epoch, m.record_save_path + '/accuracy_history.pkl')
utility.save_pickle_files(val_loss_per_epoch, m.record_save_path + '/val_loss_history.pkl')
utility.save_pickle_files(val_accuracy_per_epoch, m.record_save_path + '/val_accuracy_history.pkl.pkl')

In [4]:
# evaluate model
model = m.EmotionCNN(num_classes=7)
utility.model_validation(model, device, test_loader, m.pth_save_path)
utility.model_validation(model, device, test_loader, m.pth_manual_save_path)

Test Accuracy: 57.31401504597381%
Test Accuracy: 60.810810810810814%
